In [ ]:
# -*- coding: utf-8 -*-
import pymc3 as pm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from statsmodels.stats.outliers_influence import summary_table
import itertools
import math
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
import arviz as az
from prettytable import PrettyTable
from prettytable import from_csv
#%% UPSCALING
def TYpe_Val(df,d1,d2):
    SFou = dict()
    for int_sfou in range(1,3):
        if int_sfou == 1:
            SFou[int_sfou] = df[df['Area'] >= d1]
        if int_sfou != 1:
            SFou[int_sfou] = SFou[1] [SFou[1]['Area'] < d2]
    ClasSFI = SFou[2] 
    return ClasSFI
def Tt_Valiae(dv,m1,m2):
    SmTe = dict()
    for int_SmTe in range(1,3):
        if int_SmTe == 1:
            SmTe[int_SmTe] = dv[dv['Chla'] >= m1]
        if int_SmTe != 1:
            SmTe[int_SmTe] = SmTe[1] [SmTe[1]['Chla'] < m2]
    ClasSmTe = SmTe[2] 
    return ClasSmTe
def Acess_TVtt(df,d1,d2):
    dv = TYpe_Val(df,d1,d2)
    ETRR = dict()
    for kenY in range(1,5):
        if kenY < 2:
            ETRR[kenY] = dv[dv['Chla']<2]
        if kenY == 2 :
            ETRR[kenY] = Tt_Valiae(dv,2,7)
        if kenY == 3:
            ETRR[kenY] = Tt_Valiae(dv,7,30)
        if kenY > 3:
            ETRR[kenY] = dv[dv['Chla']>= 30]
    if len(ETRR[2]) == 0:
        ETRR[2] = ETRR[1]
    return ETRR
def MCMC(data):
    with pm.Model():
        Means = pm.Uniform("Means",0,10)
        Eps = pm.Uniform("Eps",0,20)
        y = pm.Normal("y", mu= Means, sigma=Eps, observed=data)
        trace= pm.sample(1500, tune=1500, target_accept=0.9, return_inferencedata=True)  ## Return the values of "mu_mean"
    az.plot_trace(trace, var_names=["Means", "Eps"]);
    result_MCMC = pm.summary(trace)
    theMean= result_MCMC['mean']['Means'] 
    theEPS= result_MCMC['mean']['Eps'] 
    return theMean,theEPS
def Qs_enter(dataframe,ges,method):
    er_N_dp = dict()
    for int_erndp in range(1,5):
        ResoF = MCMC(dataframe[int_erndp ][ges])
        Res_mean = ResoF[0]
        Res_STD = ResoF[1]/ ( len(dataframe[int_erndp ][ges])**(1/2))
        if method == 'M':
            er_N_dp[int_erndp] =  Res_mean
        if method == 'Z':
            er_N_dp[int_erndp] =  Res_mean - (1.96 * Res_STD )    
        if method == 'Y':
            er_N_dp[int_erndp] =  Res_mean + (1.96 * Res_STD )      
    return er_N_dp
def NeCd_K(dataframe,valueframe,depth):
    ii_N_dp = dict()
    for iteree in range(1,5):
        ii_N_dp[iteree] = dataframe[iteree]*365*(valueframe[iteree] * 1000000 * depth )/1000000000000 
    return ii_N_dp
def Gusuan_plot2(area,depth,df,d1,d2,method):
    #SA = area
    Value_Chla = dict()
    intVflo = [0.1422,0.3716,0.3027,0.1835]
    for num_vca in range(1,5):
        Value_Chla[num_vca] = intVflo[num_vca - 1] * area
    ENTER_PP = Acess_TVtt(df,d1,d2)    
    er_N_dp = Qs_enter(ENTER_PP,'N_DE',method);
    er_P_dp = Qs_enter(ENTER_PP,'P_DE',method); 
    er_N_en = Qs_enter(ENTER_PP,'N_EN',method);  
    er_P_en = Qs_enter(ENTER_PP,'P_EN',method); 
    ii_N_dp = NeCd_K(er_N_dp,Value_Chla,depth)
    ii_P_dp = NeCd_K(er_P_dp,Value_Chla,depth)
    ii_N_en = NeCd_K(er_N_en,Value_Chla,depth)
    ii_P_en = NeCd_K(er_P_en,Value_Chla,depth)            
    return ii_N_dp,ii_N_en,ii_P_dp,ii_P_en
def DetectQ(method):
    Qa = Gusuan_plot2(539830,0.5,df,0.1,1,method)
    Qb = Gusuan_plot2(477540,5.4,df,1,10,method)
    Qc = Gusuan_plot2(423002,10.4,df,10,100,method)
    Qd = Gusuan_plot2(1704431,63,df,100,1000,method)
    eQ_EEN = (*Qa,*Qb,*Qc,*Qd)
    return eQ_EEN
def OutRES(method):
    dataWR= DetectQ(method)
    Toee_res = dict()
    for Int_res in range(1,17):
            Toee_res[Int_res] = np.round( sum(dataWR[Int_res - 1].values()) , 2)
    return Toee_res


if __name__ ==  '__main__':
    ## Run PYMC3(MCMC) Output Result
    filepathQ = r'..\NLAmodel';
    eEdata = pd.read_csv(filepathQ + '\\NLAmodel_output.csv')
    df = eEdata ;left = OutRES('Z');mid = OutRES('M');right = OutRES('Y');left = list(left.values()); mid = list(mid.values());right = list(right.values())
    dall = []
    for inters in range(0,4):
        d_attrex =  [str(mid[0 + inters*4 ]) +'(' + str(left[0 +  inters*4 ]) + ',' + str(right[0  + inters*4]) + ')',
               str(mid[2  + inters*4 ]) +'(' + str(left[2 + inters*4]) + ',' + str(right[2 + inters*4]) + ')',
               str(mid[1 + inters*4]) +'(' + str(left[1 + inters*4]) + ',' + str(right[1 + inters*4]) + ')',
               str(mid[3+ inters*4]) +'(' + str(left[3 + inters*4]) + ',' + str(right[3 + inters*4]) + ')']
        dall.append(d_attrex)
    table = pd.DataFrame({'Chla class(ug/L)':['N_Depletion','P_Depletion','N_Enrichment','P_Enrichment'],'<2 (Oligotrophic)':dall[0],
        '2-7 (Mesotrophic)': dall[1],   '7-30 (Eutrophic)':dall[2] ,  '>30 (Hypereutrophic)':dall[3] });table.index = ['Ⅰ','Ⅱ','Ⅲ','Ⅳ']
    table.to_csv(r'..\NLAmodel\Res_print.csv',index=False);fp = open(r'..\NLAmodel\Res_print.csv', "r") 
    tb = from_csv(fp);print (tb)

#%%


In [ ]:
print (tb)

Due to the random characteristics of MCMC, the results may be different from those in the Manuscript, and if there are few samples, the results of MCMC will have large errors. For a small number of samples, it can usually be replaced by the mean and standard deviation in the group.